In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow  as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

In [2]:
## Creating five_fold_validation test
def five_fold_split(X,Y):
    step = (len(X)//5)
    E, L, ET, LT = [],[],[],[]
    indexes = set(list(range(len(X))))
    for i in range(5):
        if i < 4:
            LT.append(Y[i*step:(i+1)*step])
            ET.append(X[i*step:(i+1)*step])
            E.append(X[list(indexes.difference(list(range(i*step,(i+1)*step))))])
            L.append(Y[list(indexes.difference(list(range(i*step,(i+1)*step))))])
        else:
            LT.append(Y[i*step:])
            ET.append(X[i*step:])
            E.append(X[:i*step])
            L.append(Y[:i*step])

    return E,L, ET, LT

In [4]:
#Acquiring the dataset from CSV file
df = pd.read_csv('../spam.csv',delimiter=',',encoding='latin-1')
df.head()

#Dropping last three columns, (since the dataset has the last three columns filled with nan values)
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

labels = df.v1.to_numpy()
ham = sum(labels=="ham")
print("Ham: " +  str(ham)+ " Spam: " + str(5572-ham))

X = df.v2
Y = df.v1
#Encoding labels
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
#Splitting the dataset into train and test set (85%-15%)
E, L, ET, LT = five_fold_split(X,Y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
Ham: 4825 Spam: 747


In [5]:
max_words = 1000
max_len = 150

Etokseq, ETtokseq = [],[]
for fold in range(5):
    tok = Tokenizer(num_words=max_words)
    seq = tok.fit_on_texts(E[fold])
    sequencesTrain = tok.texts_to_sequences(E[fold])
    sequencesTest = tok.texts_to_sequences(ET[fold])
    Etokseq.append(sequence.pad_sequences(sequencesTrain,maxlen=max_len))
    ETtokseq.append(sequence.pad_sequences(sequencesTest,maxlen=max_len))

In [6]:
optimizers = [tf.keras.optimizers.RMSprop, tf.keras.optimizers.Adam]
losses = [tf.keras.losses.MSE, tf.keras.losses.binary_crossentropy]
batch_sizes = [16,32,64,128]
learning_rates = [0.005, 0.001, 0.0005]

In [11]:
def RNN_std():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [7]:
def RNN2():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,25,input_length=max_len)(inputs)
    layer = LSTM(32)(layer)
    layer = Dense(64,name='FC1')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(128,activation='relu',name='FC2')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(32,activation='relu',name='FC3')(layer)
    layer = Dropout(0.1)(layer)
    layer = Dense(1,activation='relu',name='FC4')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [14]:
def RNN3():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,100,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dropout(0.1)(layer)
    layer = Dense(1,activation='relu',name='FC2')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [15]:
results = []

In [16]:
import numpy as np
model = None
with open('resultsDLARNN3', 'w') as f:
    for optimizer in optimizers:
        for loss in losses:
            for bs in batch_sizes:
                for lr in learning_rates:
                    train_accuracies = np.zeros(5)
                    test_accuracies = np.zeros(5)
                    for fold in range(5):
                        model = RNN3()
                        model.compile(loss=loss,optimizer=optimizer(learning_rate=lr),metrics=['accuracy'])
                        history = model.fit(Etokseq[fold],L[fold],batch_size=bs,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
                        train_accuracies[fold] = history.history['accuracy'][-1]
                        test_accuracies[fold] = model.evaluate(ETtokseq[fold],LT[fold])[1]
                        del model
                    mean_train = np.mean(train_accuracies)
                    mean_test = np.mean(test_accuracies)
                    string = "BatchSize: " + str(bs) + " Loss:" + str(loss) + " Optimizer:" + str(optimizer)+ " LearningRate: " + str(lr) + " TrainAcc: " + str(mean_train) + " TestAcc: " + str(mean_test) +"\n"
                    print(string)
                    results.append(string)
                    f.write(string)

Epoch 1/10
223/223 [==============================] - 3s 14ms/step - loss: 0.0986 - accuracy: 0.8996 - val_loss: 0.1289 - val_accuracy: 0.8711
Epoch 2/10
35/35 [==============================] - 0s 7ms/step - loss: 0.1508 - accuracy: 0.8492
Epoch 1/10
223/223 [==============================] - 3s 14ms/step - loss: 0.0444 - accuracy: 0.9571 - val_loss: 0.0164 - val_accuracy: 0.9787
Epoch 2/10
223/223 [==============================] - 3s 13ms/step - loss: 0.0154 - accuracy: 0.9837 - val_loss: 0.0112 - val_accuracy: 0.9888
Epoch 3/10
35/35 [==============================] - 0s 8ms/step - loss: 0.0154 - accuracy: 0.9811
Epoch 1/10
223/223 [==============================] - 3s 14ms/step - loss: 0.0650 - accuracy: 0.9321 - val_loss: 0.0181 - val_accuracy: 0.9821
Epoch 2/10
35/35 [==============================] - 0s 8ms/step - loss: 0.0237 - accuracy: 0.9731
Epoch 1/10
223/223 [==============================] - 3s 14ms/step - loss: 0.1335 - accuracy: 0.8665 - val_loss: 0.1289 - val_accuracy

112/112 [==============================] - 2s 20ms/step - loss: 0.0326 - accuracy: 0.9641 - val_loss: 0.0153 - val_accuracy: 0.9854
Epoch 2/10
112/112 [==============================] - 2s 18ms/step - loss: 0.0123 - accuracy: 0.9877 - val_loss: 0.0109 - val_accuracy: 0.9899
Epoch 3/10
35/35 [==============================] - 0s 7ms/step - loss: 0.0184 - accuracy: 0.9820
Epoch 1/10
112/112 [==============================] - 2s 20ms/step - loss: 0.0402 - accuracy: 0.9560 - val_loss: 0.0210 - val_accuracy: 0.9798
Epoch 2/10
112/112 [==============================] - 2s 18ms/step - loss: 0.0166 - accuracy: 0.9826 - val_loss: 0.0141 - val_accuracy: 0.9843
Epoch 3/10
112/112 [==============================] - 2s 18ms/step - loss: 0.0103 - accuracy: 0.9891 - val_loss: 0.0100 - val_accuracy: 0.9899
Epoch 4/10
35/35 [==============================] - 0s 7ms/step - loss: 0.0128 - accuracy: 0.9847
Epoch 1/10
112/112 [==============================] - 2s 20ms/step - loss: 0.1379 - accuracy: 0.8620

112/112 [==============================] - 2s 17ms/step - loss: 0.0164 - accuracy: 0.9835 - val_loss: 0.0144 - val_accuracy: 0.9843
Epoch 3/10
35/35 [==============================] - 0s 8ms/step - loss: 0.0218 - accuracy: 0.9785
Epoch 1/10
112/112 [==============================] - 2s 20ms/step - loss: 0.0444 - accuracy: 0.9467 - val_loss: 0.0209 - val_accuracy: 0.9753
Epoch 2/10
112/112 [==============================] - 2s 17ms/step - loss: 0.0160 - accuracy: 0.9837 - val_loss: 0.0159 - val_accuracy: 0.9809
Epoch 3/10
35/35 [==============================] - 0s 7ms/step - loss: 0.0178 - accuracy: 0.9785
BatchSize: 32 Loss:<function mean_squared_error at 0x7f76fddf7bf8> Optimizer:<class 'tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop'> LearningRate: 0.0005 TrainAcc: 0.9899032950401306 TestAcc: 0.9820544123649597

Epoch 1/10
56/56 [==============================] - 2s 34ms/step - loss: 0.0939 - accuracy: 0.9038 - val_loss: 0.0231 - val_accuracy: 0.9776
Epoch 2/10
56/56 [========

56/56 [==============================] - 2s 33ms/step - loss: 0.0544 - accuracy: 0.9327 - val_loss: 0.0179 - val_accuracy: 0.9798
Epoch 2/10
56/56 [==============================] - 2s 28ms/step - loss: 0.0181 - accuracy: 0.9832 - val_loss: 0.0172 - val_accuracy: 0.9854
Epoch 3/10
56/56 [==============================] - 2s 29ms/step - loss: 0.0140 - accuracy: 0.9877 - val_loss: 0.0146 - val_accuracy: 0.9854
Epoch 4/10
56/56 [==============================] - 2s 28ms/step - loss: 0.0120 - accuracy: 0.9891 - val_loss: 0.0142 - val_accuracy: 0.9821
Epoch 5/10
56/56 [==============================] - 2s 29ms/step - loss: 0.0100 - accuracy: 0.9902 - val_loss: 0.0130 - val_accuracy: 0.9832
Epoch 6/10
56/56 [==============================] - 2s 29ms/step - loss: 0.0086 - accuracy: 0.9921 - val_loss: 0.0126 - val_accuracy: 0.9843
Epoch 7/10
35/35 [==============================] - 0s 7ms/step - loss: 0.0136 - accuracy: 0.9874
Epoch 1/10
56/56 [==============================] - 2s 33ms/step - 

Epoch 1/10
28/28 [==============================] - 2s 54ms/step - loss: 0.0703 - accuracy: 0.9080 - val_loss: 0.0276 - val_accuracy: 0.9675
Epoch 2/10
28/28 [==============================] - 1s 44ms/step - loss: 0.0213 - accuracy: 0.9792 - val_loss: 0.0171 - val_accuracy: 0.9854
Epoch 3/10
28/28 [==============================] - 1s 45ms/step - loss: 0.0155 - accuracy: 0.9849 - val_loss: 0.0149 - val_accuracy: 0.9832
Epoch 4/10
28/28 [==============================] - 1s 44ms/step - loss: 0.0128 - accuracy: 0.9877 - val_loss: 0.0145 - val_accuracy: 0.9832
Epoch 5/10
35/35 [==============================] - 0s 8ms/step - loss: 0.0170 - accuracy: 0.9865
Epoch 1/10
28/28 [==============================] - 2s 62ms/step - loss: 0.0747 - accuracy: 0.9007 - val_loss: 0.0252 - val_accuracy: 0.9765
Epoch 2/10
28/28 [==============================] - 1s 44ms/step - loss: 0.0237 - accuracy: 0.9764 - val_loss: 0.0168 - val_accuracy: 0.9798
Epoch 3/10
28/28 [==============================] - 1s 4

...

Epoch 3/10
28/28 [==============================] - 1s 42ms/step - loss: 0.0100 - accuracy: 0.9910 - val_loss: 0.0139 - val_accuracy: 0.9832
Epoch 4/10
35/35 [==============================] - 0s 7ms/step - loss: 0.0196 - accuracy: 0.9785
Epoch 1/10
28/28 [==============================] - 4s 160ms/step - loss: 0.0682 - accuracy: 0.9144 - val_loss: 0.0222 - val_accuracy: 0.9753
Epoch 2/10
28/28 [==============================] - 1s 43ms/step - loss: 0.0165 - accuracy: 0.9834 - val_loss: 0.0183 - val_accuracy: 0.9798
Epoch 3/10
28/28 [==============================] - 1s 43ms/step - loss: 0.0104 - accuracy: 0.9891 - val_loss: 0.0169 - val_accuracy: 0.9776
Epoch 4/10
35/35 [==============================] - 0s 7ms/step - loss: 0.0167 - accuracy: 0.9812
BatchSize: 128 Loss:<function mean_squared_error at 0x7f76fddf7bf8> Optimizer:<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'> LearningRate: 0.001 TrainAcc: 0.9931568622589111 TestAcc: 0.9829511046409607

Epoch 1/10
28/28 [========

Epoch 1/10
223/223 [==============================] - 3s 14ms/step - loss: 2.0071 - accuracy: 0.8699 - val_loss: 1.9886 - val_accuracy: 0.8711
Epoch 2/10
35/35 [==============================] - 0s 7ms/step - loss: 2.3262 - accuracy: 0.8492
Epoch 1/10
223/223 [==============================] - 3s 14ms/step - loss: 0.2019 - accuracy: 0.9456 - val_loss: 0.1522 - val_accuracy: 0.9787
Epoch 2/10
223/223 [==============================] - 3s 14ms/step - loss: 0.0885 - accuracy: 0.9893 - val_loss: 0.1032 - val_accuracy: 0.9888
Epoch 3/10
35/35 [==============================] - 0s 7ms/step - loss: 0.1102 - accuracy: 0.9811
Epoch 1/10
223/223 [==============================] - 3s 14ms/step - loss: 0.2131 - accuracy: 0.9546 - val_loss: 0.2080 - val_accuracy: 0.9709
Epoch 2/10
223/223 [==============================] - 3s 13ms/step - loss: 0.0786 - accuracy: 0.9885 - val_loss: 0.1184 - val_accuracy: 0.9877
Epoch 3/10
35/35 [==============================] - 0s 7ms/step - loss: 0.1445 - accuracy

112/112 [==============================] - 2s 20ms/step - loss: 0.2571 - accuracy: 0.9019 - val_loss: 0.0846 - val_accuracy: 0.9888
Epoch 2/10
35/35 [==============================] - 0s 6ms/step - loss: 0.0785 - accuracy: 0.9874
Epoch 1/10
112/112 [==============================] - 2s 20ms/step - loss: 0.2242 - accuracy: 0.9307 - val_loss: 0.1274 - val_accuracy: 0.9877
Epoch 2/10
112/112 [==============================] - 2s 17ms/step - loss: 0.0746 - accuracy: 0.9857 - val_loss: 0.1198 - val_accuracy: 0.9865
Epoch 3/10
112/112 [==============================] - 2s 17ms/step - loss: 0.0465 - accuracy: 0.9930 - val_loss: 0.1148 - val_accuracy: 0.9877
Epoch 4/10
35/35 [==============================] - 0s 7ms/step - loss: 0.2345 - accuracy: 0.9820
Epoch 1/10
112/112 [==============================] - 2s 20ms/step - loss: 2.0590 - accuracy: 0.8665 - val_loss: 1.9886 - val_accuracy: 0.8711
Epoch 2/10
35/35 [==============================] - 0s 7ms/step - loss: 2.1600 - accuracy: 0.8600
Ep

35/35 [==============================] - 0s 7ms/step - loss: 0.1219 - accuracy: 0.9829
Epoch 1/10
56/56 [==============================] - 2s 32ms/step - loss: 0.2478 - accuracy: 0.9229 - val_loss: 0.1101 - val_accuracy: 0.9832
Epoch 2/10
56/56 [==============================] - 2s 27ms/step - loss: 0.0553 - accuracy: 0.9865 - val_loss: 0.1058 - val_accuracy: 0.9832
Epoch 3/10
56/56 [==============================] - 2s 27ms/step - loss: 0.0391 - accuracy: 0.9916 - val_loss: 0.0893 - val_accuracy: 0.9865
Epoch 4/10
56/56 [==============================] - 2s 27ms/step - loss: 0.0298 - accuracy: 0.9947 - val_loss: 0.0875 - val_accuracy: 0.9888
Epoch 5/10
35/35 [==============================] - 0s 7ms/step - loss: 0.1276 - accuracy: 0.9847
Epoch 1/10
56/56 [==============================] - 2s 32ms/step - loss: 0.2711 - accuracy: 0.9088 - val_loss: 0.1481 - val_accuracy: 0.9350
Epoch 2/10
56/56 [==============================] - 2s 27ms/step - loss: 0.0819 - accuracy: 0.9742 - val_loss:

Epoch 2/10
28/28 [==============================] - 1s 42ms/step - loss: 0.1154 - accuracy: 0.9722 - val_loss: 0.0781 - val_accuracy: 0.9765
Epoch 3/10
28/28 [==============================] - 1s 42ms/step - loss: 0.0676 - accuracy: 0.9860 - val_loss: 0.0609 - val_accuracy: 0.9843
Epoch 4/10
28/28 [==============================] - 1s 42ms/step - loss: 0.0484 - accuracy: 0.9893 - val_loss: 0.0479 - val_accuracy: 0.9865
Epoch 5/10
35/35 [==============================] - 0s 7ms/step - loss: 0.0777 - accuracy: 0.9874
Epoch 1/10
28/28 [==============================] - 1s 53ms/step - loss: 0.3681 - accuracy: 0.8718 - val_loss: 0.1057 - val_accuracy: 0.9731
Epoch 2/10
28/28 [==============================] - 1s 43ms/step - loss: 0.0955 - accuracy: 0.9784 - val_loss: 0.0892 - val_accuracy: 0.9821
Epoch 3/10
35/35 [==============================] - 0s 7ms/step - loss: 0.1302 - accuracy: 0.9820
Epoch 1/10
28/28 [==============================] - 1s 53ms/step - loss: 0.3478 - accuracy: 0.8780 

In [12]:
del model

NameError: name 'model' is not defined

In [9]:

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score



train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = Naive.predict(Etokseq[fold])
    predictions_Test_NB = Naive.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("Naive Bayes Accuracy Train Score -> ",train_accuracies[fold])
    print("Naive Bayes Accuracy Test Score -> ",test_accuracies[fold])
    del Naive
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

Naive Bayes Accuracy Train Score ->  77.38896366083445
Naive Bayes Accuracy Test Score ->  75.31418312387792
Naive Bayes Accuracy Train Score ->  76.69358456707045
Naive Bayes Accuracy Test Score ->  78.27648114901257
Naive Bayes Accuracy Train Score ->  78.24136384028712
Naive Bayes Accuracy Test Score ->  78.45601436265709
Naive Bayes Accuracy Train Score ->  77.38896366083445
Naive Bayes Accuracy Test Score ->  77.10951526032316
Naive Bayes Accuracy Train Score ->  76.41382405745063
Naive Bayes Accuracy Test Score ->  78.2258064516129
Mean Train Accuracy: 77.22533995729543
Mean Test Accuracy: 77.47640006949673


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/loc

In [8]:
del Naive

In [13]:
from sklearn import  svm
train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    SVM = svm.SVC(C=1000, gamma=0.1)
    SVM.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = SVM.predict(Etokseq[fold])
    predictions_Test_NB = SVM.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("SVM Accuracy Train Score -> ",train_accuracies[fold])
    print("SVM Accuracy Test Score -> ",test_accuracies[fold])
    del SVM
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/loc

SVM Accuracy Train Score ->  99.9775684163302
SVM Bayes Accuracy Test Score ->  90.12567324955117
SVM Accuracy Train Score ->  99.9775684163302
SVM Bayes Accuracy Test Score ->  91.92100538599641
SVM Accuracy Train Score ->  99.9775684163302
SVM Bayes Accuracy Test Score ->  92.01077199281868
SVM Accuracy Train Score ->  100.0
SVM Bayes Accuracy Test Score ->  90.66427289048474
SVM Accuracy Train Score ->  99.97755834829444
SVM Bayes Accuracy Test Score ->  91.66666666666666
Mean Train Accuracy: 99.98205271945702
Mean Test Accuracy: 91.27767803710353


In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    KNN = KNeighborsClassifier(n_neighbors=5)
    KNN.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = KNN.predict(Etokseq[fold])
    predictions_Test_NB = KNN.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("KNN Accuracy Train Score -> ",train_accuracies[fold])
    print("KNN Accuracy Test Score -> ",test_accuracies[fold])
    del KNN
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8:

Naive Bayes Accuracy Train Score ->  91.99192462987888
Naive Bayes Accuracy Test Score ->  87.61220825852783
Naive Bayes Accuracy Train Score ->  91.13952445042621
Naive Bayes Accuracy Test Score ->  88.24057450628366
Naive Bayes Accuracy Train Score ->  91.74517720951098
Naive Bayes Accuracy Test Score ->  90.84380610412927
Naive Bayes Accuracy Train Score ->  92.21624046657693
Naive Bayes Accuracy Test Score ->  87.52244165170556
Naive Bayes Accuracy Train Score ->  91.65170556552962
Naive Bayes Accuracy Test Score ->  89.69534050179212
Mean Train Accuracy: 91.74891446438453
Mean Test Accuracy: 88.78287420448768


In [10]:
from sklearn.tree import DecisionTreeClassifier
train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    DTC = DecisionTreeClassifier()
    DTC.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = DTC.predict(Etokseq[fold])
    predictions_Test_NB = DTC.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("DTC Accuracy Train Score -> ",train_accuracies[fold])
    print("DTC Accuracy Test Score -> ",test_accuracies[fold])
    del DTC
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

DTC Accuracy Train Score ->  99.9775684163302
DTC Accuracy Test Score ->  88.50987432675045
DTC Accuracy Train Score ->  99.9775684163302
DTC Accuracy Test Score ->  89.22800718132855
DTC Accuracy Train Score ->  99.9775684163302
DTC Accuracy Test Score ->  91.11310592459606
DTC Accuracy Train Score ->  100.0
DTC Accuracy Test Score ->  87.97127468581688
DTC Accuracy Train Score ->  99.97755834829444
DTC Accuracy Test Score ->  88.88888888888889
Mean Train Accuracy: 99.98205271945702
Mean Test Accuracy: 89.14223020147617


In [11]:
from sklearn.ensemble import RandomForestClassifier
train_accuracies = np.zeros(5)
test_accuracies = np.zeros(5)
for fold in range(5):
    RFC = RandomForestClassifier()
    RFC.fit(Etokseq[fold],L[fold])
    # predict the labels on validation dataset
    predictions_Train_NB = RFC.predict(Etokseq[fold])
    predictions_Test_NB = RFC.predict(ETtokseq[fold])
    # Use accuracy_score function to get the accuracy
    train_accuracies[fold] = accuracy_score(predictions_Train_NB ,L[fold])*100
    test_accuracies[fold] = accuracy_score(predictions_Test_NB ,LT[fold])*100
    print("RFC Accuracy Train Score -> ",train_accuracies[fold])
    print("RFC Accuracy Test Score -> ",test_accuracies[fold])
    del RFC
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)


print("Mean Train Accuracy: " + str(mean_train))
print("Mean Test Accuracy: " + str(mean_test))

/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: Dat

RFC Accuracy Train Score ->  99.9775684163302
RFC Accuracy Test Score ->  92.63913824057451
RFC Accuracy Train Score ->  99.9775684163302
RFC Accuracy Test Score ->  93.35727109515261
RFC Accuracy Train Score ->  99.9775684163302
RFC Accuracy Test Score ->  94.3447037701975
RFC Accuracy Train Score ->  100.0
RFC Accuracy Test Score ->  92.36983842010773
RFC Accuracy Train Score ->  99.95511669658886
RFC Accuracy Test Score ->  93.81720430107528
Mean Train Accuracy: 99.97756438911588
Mean Test Accuracy: 93.30563116542152
